In [8]:
import os
import numpy as np
import pandas as pd
from glob import glob
import zipfile
import json
from tqdm import tqdm
from PIL import Image
import sys

from encoder import crop_aware_name_decoder

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.name

if os.name == 'posix': 
    plt.rc('font', family='AppleGothic')
elif os.name == 'nt':
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) 

%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import cv2
from glob import glob
from PIL import Image
import json
import seaborn as sns
import os
import pickle
import time
from tqdm import tqdm

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected" 

In [4]:
root = os.getcwd()
sub_train_df = pd.read_csv(root + '\\train_df.csv')
sub_valid_df = pd.read_csv(root + '\\valid_df.csv')
test_df = pd.read_csv(root + '\\test_df.csv')

In [5]:
print(f"train data size : {len(sub_train_df)}")
print(f"valid data size : {len(sub_valid_df)}")
print(f"test data size: {len(test_df)}")

train data size : 120272
valid data size : 13364
test data size: 16701


In [11]:
len(sub_train_df) +len(sub_valid_df) + len(test_df)

150337

In [13]:
sub_train_df.head()

,path,date,worker,height,width,task,types,region,disease,crop,area,grow,risk,xl,yl,xr,yr
0,V006_79_0_00_03_03_12_0_c31_20201027_0012_S01_...,2020/10/27,NaN,3492,4656,79,0,NaN,0,3,3,12,0,900,193,4656,3229
1,V006_79_0_00_09_03_12_0_c32_20201223_0480_S01_...,2020/12/23,NaN,4160,3120,79,0,NaN,0,9,3,12,0,1600,1565,2002,3285
2,V006_79_0_00_09_03_12_0_c27_20201216_0020_S01_...,2020/12/16,NaN,4032,2268,79,0,NaN,0,9,3,12,0,809,1381,1314,2741
3,V006_79_0_00_05_03_13_0_c10_20201221_0264_S01_...,2020/12/21,NaN,4032,3024,79,0,NaN,0,5,3,13,0,44,778,3024,3446
4,V006_79_0_00_07_01_13_0_c06_20201211_0003_S01_...,2020/12/11,NaN,3024,3024,79,0,NaN,0,7,1,13,0,543,907,2556,2557
